# ほかの市場で値動きがあった後にS&P500先物がどう動くか

「E-MiniS&P500-60min-EDA-02-inter-market-corr-analysis.ipynb」では、60分足で検証したが、単純に先行指標としてつかえる商品は見つからなかった。60分足では時間枠が短すぎた可能性があるため、日足データで同じ検証をしてみる。

## 目的
S&P500先物の先行指標となりうる商品を探すこと

### 仮説
- 金利が上がったらS&P500も上がる。
- 原油や金が上がったらS&P500は下がる。
- ダウとS&P500が異なる動きをしたらその後修正の方向に価格が動く。

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import scipy
import scipy.stats as st
import statsmodels.stats.anova as anova
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import datetime as dt
from dateutil.relativedelta import relativedelta
import locale
from joblib import Parallel, delayed

import finalib as fl
import finalib.mine as mi
import ta

# 月や曜日を英語で取得するためこの設定をしておく
locale.setlocale(locale.LC_TIME, 'en_US.UTF-8')

'en_US.UTF-8'

# データ読み込み

In [3]:
# 生データ

# 時間枠
time_scale = 'daily'

sp_dir = 'data/e-mini-sp500-200530'
df_sp_raw = pd.read_csv(f'{sp_dir}/e-mini-sp500-{time_scale}.csv')

dow_dir = 'data/e-mini-dow-200626'
df_dow_raw = pd.read_csv(f'{dow_dir}/e-mini-dow-{time_scale}.csv')

oil_dir = 'data/e-mini-crude-oil-200626'
df_oil_raw = pd.read_csv(f'{oil_dir}/e-mini-crude-oil-{time_scale}.csv')

gold_dir = 'data/gold-200626'
df_gold_raw = pd.read_csv(f'{gold_dir}/gold-{time_scale}.csv')

tnote2_dir = 'data/t-notes-2yr-200626'
df_tnote2_raw = pd.read_csv(f'{tnote2_dir}/t-notes-2yr-{time_scale}.csv')

tnote5_dir = 'data/t-notes-5yr-200626'
df_tnote5_raw = pd.read_csv(f'{tnote5_dir}/t-notes-5yr-{time_scale}.csv')

tnote10_dir = 'data/t-notes-10yr-200626'
df_tnote10_raw = pd.read_csv(f'{tnote10_dir}/t-notes-10yr-{time_scale}.csv')

tbond30_dir = 'data/t-bond-30yr-200629'
df_tbond30_raw = pd.read_csv(f'{tbond30_dir}/t-bond-30yr-{time_scale}.csv')

dfnames = ['sp', 'dow', 'oil', 'gold', 'tnote2', 'tnote5', 'tnote10', 'tbond30']
dfraws = [df_sp_raw, df_dow_raw, df_oil_raw, df_gold_raw, df_tnote2_raw, df_tnote5_raw, df_tnote10_raw, df_tbond30_raw]
#dfraws = [df_sp_raw, df_dow_raw, df_oil_raw, df_gold_raw]
#dfraws = [df_sp_raw, df_dow_raw]

In [7]:
dfraws[0].head(1)

,Date,Time,Open,High,Low,Close,Vol,OI
0,09/11/1997,17:00,1071.25,1082.25,1062.75,1068.5,11825,2909


In [5]:
dfs = []
for df in dfraws:
    dfs.append(df.copy())

In [6]:
# 日足だから時間は不要だけど、入っていても特に問題ないはず
def to_datetime_idx(df):
    df['DT'] = (df['Date'] + '-' + df['Time']).map(lambda s: dt.datetime.strptime(s, '%m/%d/%Y-%H:%M'))
    return df.set_index('DT')

for i, df in enumerate(dfs):
    dfs[i] = to_datetime_idx(df)

# ある商品が上がったらその後S&P500先物が上がるか

## 検証１
- （１）S&P500先物が上昇する確率
- （２）ある商品が上昇した次の足でS&P500先物が上昇する確率
- （２）が（１）より大きければ（統計的な判断は今はしない）その商品が正方向の先行指標として使える可能性がある。小さければ負方向の指標として使える可能性がある。

### 結果
- 60分足の時と同様の結果。
- （２）が（１）より大きいと言える商品はなかった。一番大きかったのが`P(S&P UP | tnote10 UP) = 0.5524384980578334`だが、事前確率からの上昇は0.7%程度であり、ここにもし有意な差があったとしても、予測には使えない。
- 逆方向、つまり下落の予測についてもほぼ同じ結果になるはずのため試す必要はない。

In [15]:
# 値動きの列を加える
# 値動き　＝　終値　－　始値
# 上昇　＝　値動き　＞　０
# とする。

CODIFF_COL = 'CODiff'
ISUP_COL = 'isUp'

def add_CODiff(df):
    df[CODIFF_COL] = df['Close'] - df['Open']
    return df

def add_isUp(df):
    df[ISUP_COL] = df[CODIFF_COL] > 0
    return df

for i in range(len(dfs)):
    dfs[i] = add_CODiff(dfs[i])
    dfs[i] = add_isUp(dfs[i])

In [30]:
# 各商品の上昇列を取ってきて一つのDFにする
isUp_df = dfs[0].loc[:, [ISUP_COL]].rename(columns={ISUP_COL:dfnames[0]})
for i in range(1, len(dfs)):
    isUp_df = isUp_df.merge(dfs[i].loc[:, [ISUP_COL]].rename(columns={ISUP_COL:dfnames[i]}), how='outer', left_index=True, right_index=True)

isUp_df.shape

(5774, 8)

In [31]:
# NaNを含む行を全部削除
isUp_df = isUp_df[isUp_df.notnull().all(axis=1)]
isUp_df.shape

(4479, 8)

In [32]:
# ほかの列で上昇したときに、次の足で'sp'が上昇するかどうかを調べたい
# だから、'sp'の列だけ１つ繰り上げる（これで'sp'列については日時インデックスがずれることになることに注意）
isUp_df['sp'] = isUp_df['sp'].iloc[1:].append(pd.Series([np.nan])).to_numpy()
isUp_df = isUp_df[isUp_df.notnull().all(axis=1)]
isUp_len = isUp_df.shape[0]
isUp_df.shape

(4478, 8)

In [33]:
# S&P500先物の上昇確率
print(f"P(S&P UP) = {isUp_df['sp'].sum() / isUp_len}")

P(S&P UP) = 0.5455560518088433


In [34]:
# ほかの商品が上昇した次の足でS&Pが上昇する確率
for col_name in isUp_df.columns.tolist()[1:]:
    ups_tmp = isUp_df[isUp_df[col_name]]
    print(f"P({col_name} UP) = {ups_tmp.shape[0] / isUp_len}")
    print(f"P(S&P UP | {col_name} UP) = {ups_tmp['sp'].sum() / ups_tmp.shape[0]}")

P(dow UP) = 0.541313086199196
P(S&P UP | dow UP) = 0.5334158415841584
P(oil UP) = 0.5111656989727557
P(S&P UP | oil UP) = 0.5430318916557448
P(gold UP) = 0.5129522108083966
P(S&P UP | gold UP) = 0.5446234218545929
P(tnote2 UP) = 0.47856185797230905
P(S&P UP | tnote2 UP) = 0.5482967802146523
P(tnote5 UP) = 0.5107190710138455
P(S&P UP | tnote5 UP) = 0.5505028421512899
P(tnote10 UP) = 0.5174184903974989
P(S&P UP | tnote10 UP) = 0.5524384980578334
P(tbond30 UP) = 0.5207682000893256
P(S&P UP | tbond30 UP) = 0.5493138936535163


## 検証2
ある商品が上がった足でS&Pは下がったとき、次の足でS&Pが上昇する確率が高まるような商品があるか。

- （１）下落した次の足でS&Pが上昇する確率
- （２）ある商品が上がってかつS&Pが下落したとき、次の足でS&Pが上昇する確率
- （２）が（１）より大きければ、その商品をこの条件における先行指標として使える可能性がある。

### 結果
- これについても、先行指標として使えそうなものはない。
- 一番大きかったのが`P(S&P UP | (oil UP, S&P DOWN)) = 0.5697896749521989`だが、事前確率からの上昇は0.5%程度。


In [35]:
# もう一足繰り上げたS&P列を追加
# したがってこの列はインデックスの日付より2日先の価格ということになる
isUp_df['sp0'] = isUp_df['sp'].iloc[1:].append(pd.Series([np.nan])).to_numpy()
isUp_df = isUp_df[isUp_df.notnull().all(axis=1)]
isUp_len = isUp_df.shape[0]
isUp_df.shape

(4477, 9)

In [37]:
# S&Pが下落した次の日に上昇する確率
prev_down = isUp_df[isUp_df['sp'] == False]
print(f"P(S&P UP | S&P Prev DOWN) = {prev_down['sp0'].sum() / prev_down.shape[0]}")

P(S&P UP | S&P Prev DOWN) = 0.5646191646191646


In [42]:
# ほかの商品が上がってS&Pが下がったとき、次の足でS&Pが上がる確率
for col_name in isUp_df.columns.tolist()[1:-1]:
    ups_tmp = prev_down[prev_down[col_name]]
    print(f"P({col_name} UP, S&P DOWN) = {ups_tmp.shape[0] / prev_down.shape[0]}")
    print(f"P(S&P UP | ({col_name} UP, S&P DOWN)) = {ups_tmp['sp0'].sum() / ups_tmp.shape[0]}")

P(dow UP, S&P DOWN) = 0.5557739557739557
P(S&P UP | (dow UP, S&P DOWN)) = 0.5570291777188329
P(oil UP, S&P DOWN) = 0.514004914004914
P(S&P UP | (oil UP, S&P DOWN)) = 0.5697896749521989
P(gold UP, S&P DOWN) = 0.514004914004914
P(S&P UP | (gold UP, S&P DOWN)) = 0.5592734225621415
P(tnote2 UP, S&P DOWN) = 0.4756756756756757
P(S&P UP | (tnote2 UP, S&P DOWN)) = 0.5516528925619835
P(tnote5 UP, S&P DOWN) = 0.5051597051597052
P(S&P UP | (tnote5 UP, S&P DOWN)) = 0.5554474708171206
P(tnote10 UP, S&P DOWN) = 0.5095823095823095
P(S&P UP | (tnote10 UP, S&P DOWN)) = 0.5564127290260367
P(tbond30 UP, S&P DOWN) = 0.5164619164619164
P(S&P UP | (tbond30 UP, S&P DOWN)) = 0.5528068506184586


## 検証3：検証1と検証2の結果を組み合わせたらどうなるか
- 検証1より、10年国債上昇の次の足でS&Pの上昇確率は0.7%程度上昇する
- 検証2より、原油上昇かつS&P下落の次の足でS&Pの上昇確率は0.5%程度上昇する

以下を検証
- （１）S&P下落の次の足でのS&Pの上昇確率
- （２）10年国債上昇かつ原油上昇かつS&P下落の次の足でのS&Pの上昇確率
- （２）が（１）より大きければ、買いのセットアップとして利用できる可能性がわずかにあるかも。

### 結果
- 組み合わせたら、むしろ上昇確率は下がった。この組み合わせは指標としては使えない。かなり絞り込んで母数がもとの10%まで減ったのにこの結果。

In [43]:
# （１）については検証2で計算済み

In [45]:
# （２）の確率
prev_10yrU_oilU_spD = prev_down[prev_down['tnote10'] & prev_down['oil']]
print(f"P(tnote10 UP, oil UP, sp DOWN) = {prev_10yrU_oilU_spD.shape[0] / isUp_len}")
print(f"P(sp UP | Prev (tnote10 UP, oil UP, sp DOWN)) = {prev_10yrU_oilU_spD['sp0'].sum() / prev_10yrU_oilU_spD.shape[0]}")

P(tnote10 UP, oil UP, sp DOWN) = 0.1029707393343757
P(sp UP | Prev (tnote10 UP, oil UP, sp DOWN)) = 0.5466377440347071
